In [1]:
struct Type_node
    type_node::Symbol
    children::Vector{Type_node}
end

function length(node::Type_node)
    isempty(node.children) ? 1 : sum(child -> length(child), node.children)
end

function depth(node::Type_node)
    isempty(node.children) && return 1
    return 1 + maximum(depth.(node.children))
end


depth (generic function with 1 method)

In [ ]:
using OptimizationProblems
using ADNLPModels
using Symbolics
using ExpressionTreeForge # A bit outdated so it's not easy to find a compatible version.
using CSV, DataFrames


# https://jso.dev/ExpressionTreeForge.jl/dev/tutorial/ Link to ExpressionTreeForge Tutorial

# Data from OptimizationProblems.jl
# Careful, nvar and ncon are the default, they may vary if variable_nvar or variable_ncon are true.
meta = OptimizationProblems.meta
problem_names = meta[(meta.contype .== :unconstrained) .& (.!meta.has_bounds) .& (meta.nvar .>= 5),:name]

problems = [Meta.parse("OptimizationProblems.ADNLPProblems.$(problem)") 
                for problem ∈ problem_names];

filename = "../metadata/length_and_depth.csv"
mkpath(dirname(filename))

df = DataFrame(
    :problem => String[],
    :length => Int[],
    :depth => Int[]
)

for problem in problems
    try
        nlp = eval(problem)()      # call the zero-arg constructor
        println("Extracting information of $(nlp.meta.name)")
        n = nlp.meta.nvar
        Symbolics.@variables x[1:n]
        fun = nlp.f(x)
        mtk_tree = Symbolics._toexpr(fun)
        expr_tree_Symbolics = transform_to_expr_tree(mtk_tree)

        tree_length = length(expr_tree_Symbolics)
        tree_depth = depth(expr_tree_Symbolics)
        push!(df, (; problem = nlp.meta.name, tree_length, tree_depth))

        CSV.write(
            filename,
            DataFrame([last(df)]);
            append = isfile(filename) && filesize(filename) > 0,
        )
    catch e
        @info "Failed to extract information on $(problem): $e"
        @info "Failed to extract information on $(problem): $e"
    end
end














ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


Extracting information of NZF1


ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Failed to extract information on OptimizationProblems.ADNLPProblems.NZF1: MethodError(length, (+
│   ^2
│     +
│       -
│         x[20]
│       x[7]
│   ^2
│     +
│       -
│         x[21]
│       x[8]
│   ^2
│     +
│       -
│         x[22]
│       x[9]
│   ^2
│     +
│       -
│         x[23]
│       x[10]
│   ^2
│     +
│       -
│         x[24]
│       x[11]
│   ^2
│     +
│       -
│         x[25]
│       x[12]
│   ^2
│     +
│       -60
│       *
│         1//10
│         ^2
│           +
│             -
│               x[3]
│             x[2]
│       *
│         3
│         x[1]
│   ^2
│     +
│       -60
│       *
│         1//10
│         ^2
│           +
│             -
│               x[4]
│             x[3]
│       *
│         3
│         x[2]
│   ^2
│     +
│       -60
│       *
│         1//10
│         ^2
│           +
│          

Extracting information of arglina


┌ Info: Failed to extract information on OptimizationProblems.ADNLPProblems.arglina: MethodError(length, (+
│   *
│     1//2
│     +
│       ^2
│         +
│           -1
│           -
│             *
│               1//100
│               +
│                 x[1]
│                 x[2]
│                 x[3]
│                 x[4]
│                 x[5]
│                 x[6]
│                 x[7]
│                 x[8]
│                 x[9]
│                 x[10]
│                 x[11]
│                 x[12]
│                 x[13]
│                 x[14]
│                 x[15]
│                 x[16]
│                 x[17]
│                 x[18]
│                 x[19]
│                 x[20]
│                 x[21]
│                 x[22]
│                 x[23]
│                 x[24]
│                 x[25]
│                 x[26]
│                 x[27]
│                 x[28]
│                 x[29]
│                 x[30]
│                 x[31]
│                 x[32]